!pip install implicit==0.4.4

!pip install ipywidgets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
from metrics import precision_at_k, recall_at_k

In [3]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [4]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights.item_id)
    weight = np.array(items_weights.weight)
    recs = np.random.choice(items, size=n, replace=False, p=weight)
    
    return recs.tolist()


In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
popularity = data.groupby('item_id')['sales_value'].count().reset_index().rename(columns={'sales_value': 'sales_count'})
popularity.sort_values('sales_count', ascending=False, inplace=True)
popularity['sales_count_log'] = np.log(popularity['sales_count'])
popularity['weight'] = popularity['sales_count_log'] / popularity['sales_count_log'].sum()
print(f'Сумма weight = {popularity.weight.sum()}')
popularity.head(2)

Сумма weight = 1.0


,item_id,sales_count,sales_count_log,weight
35054,1082185,27362,10.216910,0.000077
56233,6534178,18364,9.818148,0.000074


In [9]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity[['item_id', 'weight']], n=5))

result.head(2)

CPU times: user 6.84 s, sys: 50.5 ms, total: 6.89 s
Wall time: 6.92 s


,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5782017, 12171841, 306721, 1038691, 1120635]","[964135, 980263, 6773074, 820075, 15830568]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1177591, 12782120, 13095529, 10312012, 89414]","[6961847, 968036, 1108048, 7467057, 986635]"


In [10]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0006:random_recommendation
0.002:weighted_random_recommendation


По сравнению с обычной случайной рекомендацией, у взвешенной пропорционально количеству продаж, метрика качества выше.

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [11]:
%%time

top = 5000

items = popularity.item_id.unique()[:top]

popularity = popularity[:top]
popularity['weight'] = popularity['sales_count_log'] / popularity['sales_count_log'].sum()
print(f'Сумма weight = {popularity.weight.sum()}')

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity[['item_id', 'weight']], n=5))

result.head(2)

Сумма weight = 0.9999999999999999
CPU times: user 1.45 s, sys: 18 ms, total: 1.46 s
Wall time: 1.46 s


,user_id,actual,random_recommendation,weighted_random_recommendation,random_recommendation_5000,weighted_random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5782017, 12171841, 306721, 1038691, 1120635]","[964135, 980263, 6773074, 820075, 15830568]","[878026, 6633273, 946308, 1044153, 6464056]","[7025106, 996574, 883817, 849697, 9396821]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1177591, 12782120, 13095529, 10312012, 89414]","[6961847, 968036, 1108048, 7467057, 986635]","[970119, 887325, 847025, 1075707, 1131438]","[1042438, 839346, 1011645, 967476, 948420]"


In [12]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0006:random_recommendation
0.002:weighted_random_recommendation
0.0068:random_recommendation_5000
0.0076:weighted_random_recommendation_5000


В случае ТОП-5000 видим улучшение метрик в разы.

***

Теперь попробуем улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾

In [13]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [14]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [15]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666

In [16]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                  )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

print(f'Размер матрицы: {user_item_matrix.shape}')
print(f'Разреженность матрицы: {user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100}')

Размер матрицы: (2499, 5001)
Разреженность матрицы: 5.33770796861036


In [17]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [18]:
%%time

model = ItemItemRecommender(K=1, num_threads=4)


model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.05 s, sys: 26.4 ms, total: 1.08 s
Wall time: 1.08 s


In [19]:
%%time

result['own_purchases_K1'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                     model.recommend(userid=userid_to_id[x],
                                     user_items=sparse_user_item,   # на вход user-item matrix
                                     N=5,
                                     filter_already_liked_items=False,
                                     filter_items=[itemid_to_id[6666]],
                                     recalculate_user=True)])

CPU times: user 51.9 ms, sys: 2.73 ms, total: 54.6 ms
Wall time: 54 ms


In [20]:
result.head(5)

,user_id,actual,random_recommendation,weighted_random_recommendation,random_recommendation_5000,weighted_random_recommendation_5000,own_purchases_K1
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5782017, 12171841, 306721, 1038691, 1120635]","[964135, 980263, 6773074, 820075, 15830568]","[878026, 6633273, 946308, 1044153, 6464056]","[7025106, 996574, 883817, 849697, 9396821]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1177591, 12782120, 13095529, 10312012, 89414]","[6961847, 968036, 1108048, 7467057, 986635]","[970119, 887325, 847025, 1075707, 1131438]","[1042438, 839346, 1011645, 967476, 948420]","[1082185, 1098066, 6534178, 826249, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1230050, 1118768, 741007, 9270525, 987263]","[849202, 971096, 824813, 1051616, 1031831]","[925282, 1013895, 837647, 1048918, 997796]","[7410342, 9527487, 904023, 840361, 1108092]","[1082185, 981760, 995242, 1029743, 840361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5568107, 6772641, 202684, 10249623, 880635]","[16097609, 924203, 823349, 923972, 1094262]","[911411, 964968, 1000672, 868727, 1127624]","[1096036, 6533936, 9362429, 1001333, 912146]","[1082185, 995242, 1029743, 826249, 1127831]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1087601, 13218819, 863930, 13380841, 13987221]","[936037, 13512680, 6919342, 1067451, 985587]","[1081533, 1126786, 833598, 9553047, 1059969]","[988697, 845705, 8181477, 1056746, 915086]","[1082185, 981760, 995242, 1029743, 840361]"


In [22]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                  )

# user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

print(f'Размер матрицы: {user_item_matrix.shape}')
print(f'Разреженность матрицы: {user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100}')

Размер матрицы: (2499, 5001)
Разреженность матрицы: 18.23156777207984


In [23]:
%%time

# применим в модели K=3

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.05 s, sys: 17.9 ms, total: 1.07 s
Wall time: 1.06 s


In [24]:
%%time

result['own_purchases_K3'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                     model.recommend(userid=userid_to_id[x],
                                     user_items=sparse_user_item,   # на вход user-item matrix
                                     N=5,
                                     filter_already_liked_items=False,
                                     filter_items=[itemid_to_id[6666]],
                                     recalculate_user=True)])

CPU times: user 48.3 ms, sys: 1.65 ms, total: 50 ms
Wall time: 49 ms


In [26]:
%%time

# применим в модели K=5

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.04 s, sys: 9.67 ms, total: 1.05 s
Wall time: 1.04 s


In [27]:
%%time

result['own_purchases_k5'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                     model.recommend(userid=userid_to_id[x],
                                     user_items=sparse_user_item,   # на вход user-item matrix
                                     N=5,
                                     filter_already_liked_items=False,
                                     filter_items=[itemid_to_id[6666]],
                                     recalculate_user=True)])

CPU times: user 55 ms, sys: 1.15 ms, total: 56.2 ms
Wall time: 55.5 ms


In [28]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0006:random_recommendation
0.002:weighted_random_recommendation
0.0068:random_recommendation_5000
0.0076:weighted_random_recommendation_5000
0.2199:own_purchases_K1
0.3055:own_purchases_K3
0.2525:own_purchases_k5


/Users/dv/dvSync/Projects/GitHub/geekbrains/Рекомендательные системы/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


Лучший результат 0.3055. В матрице используем количество совершенных покупок (не 0/1), K=3.